In [2]:
from collections import Counter
import numpy as np
import os
import jieba
import pickle
def get_stopwords():
    stopwords=[]
    text=open(r"D:\桌面\大三上\智能信息检索导论\code\hw2\中文stopword.txt","rt",encoding='utf-8')
    f=text.read()
    f=f.replace("\n",",")
    f=f.replace(" ",',')
    stopwords.append(f.split(","))
    stopwords=stopwords[0]
    stopwords1=['the','a','in','of','on']
    for i in range(len(stopwords1)):
        stopwords.append(stopwords1[i])
    return stopwords
def filter_word(word,stopwords):
    a=1
    if len(word)<=1:
        a=0
    elif word in stopwords:
        a=0
    return a
stopwords = get_stopwords()

In [34]:
import os
import json
js1=json.load(open('base_info.json','r'))
def read_chapters(data):
    N = len(data)
    all_chapters = []
    for i in range(N):
        novel_title = js1[i]['title']
        path = r'Novels/'+novel_title
        url_dict=dict()
        with open(path+'/'+'chapters_info.csv','r',encoding='utf-8')as fr:
            for line in fr:
                line = line.strip().split(',')
                url_dict[line[0]]=line[1]
        collections = [file for file in os.listdir(path) if os.path.splitext(file)[1] == '.txt']
        for j in range(len(collections)):
            item=dict()
            item['novel']=novel_title
            item['home'] = js1[i]['url']
            file = collections[j]
            title = os.path.splitext(file)[0]
            url = url_dict[title]
            with open(os.path.join(path, file),encoding='utf-8') as fin:
                text = fin.read()
                fin.close()
            item['title']=title
            item['url']=url
            item['text']=text
            all_chapters.append(item)
    return all_chapters
all_chapters = read_chapters(js1)
all_chapters[6]

{'novel': '武道丹帝',
 'home': 'https://fanqienovel.com/page/6950489390904773646',
 'title': '第007章 魂力武技',
 'url': 'https://fanqienovel.com/reader/6950532176941154847',
 'text': '\n武道丹帝第007章 魂力武技免费阅读-番茄小说网-番茄小说旗下原创文学平台\n\n\n\nYou need to enable JavaScript to run this app.目录手机阅读第007章 魂力武技书名：武道丹帝作者：烧烤西瓜本章字数：2094字更新时间：2021-04-13 15:07:32而在此时的夜幕下……齐家，大厅！此地汇聚着齐家家主齐刀以及齐长剑等长老，但在地面之上则放着一具尸体，使得氛围一片凝重。“叶家叶炎！”“竟敢杀害我儿？”齐刀的眼眸赤红，充斥着杀意。“家主息怒，只怪老夫没能护住少爷！”齐长剑跪在地上开口。齐刀深吸了一口气道：“此事不怪你，谁能想到叶炎竟也达到了九重筑体境？我儿定是小觑了他方才被其斩杀。”“但这笔账，叶家迟早要还的。”“而且……不止叶炎一人要还！”“杀我儿一人，我让他们叶家所有少年全部陪葬。”轰！在这等杀意之下，齐家众长老噤若寒蝉。“灵台剑冢地开启，叶家也要前往吧？此次我要让叶家少年有去无回！”齐刀神色冷凝。但齐家大长老却是上前道：“家主，灵台剑冢地……叶龙定然跟随，他的实力不可忽视……”齐刀冷笑：“我何时说过我齐家要与叶龙一战？何况叶啸天的名字毕竟还写在苍云圣碑之上，我齐家怎能动手？”“这……”齐家众长老愕然。齐刀再道：“你们可别忘了，到了真正的剑冢地，我等皆是无法踏入，那里的阵法只让十八岁以下的少年进入。”齐家大长老蹙眉：“我齐家之内，齐山少爷天赋卓绝，虽早已达到了结丹境，但终究十九岁了……至于我齐家其他的少年……很难在剑冢地内将叶家之人诛杀。”叶家内，十八岁的少年不少！他们的实力可也是不弱。齐山自可横扫一切，但却无法进入啊。“杀他们，何须我齐家少年？”“用我齐家令牌，置信方家家主，告诉他……若能诛杀叶家少年我齐家将拿出一件下阶灵器！”“至于叶炎，我齐家人亲自来杀！”闻言，众长老眼眸猛然一亮。方家在这苍云城势力也极为不弱，而且方家家主的儿子方阐不满十八岁，如今已是达到

In [14]:
def chapter_jieba(data):
    term_docid_pairs = []
    abstract_length = []
    all_terms=set()#建立所有词的集合，之后做一个idf的字典方便查询计算tf—idf
    N = len(data)
    for i in range(N):
        terms = [term for term in jieba.lcut_for_search(data[i]['text'].replace(".",'').replace("-",'')) if filter_word(term,stopwords) == 1 and len(term)>1]
        # 构造term_docid_pairs
        for term in terms:
            term_docid_pairs.append((term, i))
            all_terms.add(term)
        # 统计tf_ 
        term_counts = np.array(list(Counter(terms).values()))
        log_tf = np.log(1 + term_counts) 
        # log_tf = 1+np.log(term_counts) 
        abstract_length.append(np.sqrt(np.sum(log_tf**2))) #每个向量的长度,不在这考虑idf的话，每个向量少乘以一个固定值，都一样
    return term_docid_pairs,abstract_length,all_terms
term_docid_pairs,chapter_length,all_terms = chapter_jieba(all_chapters)
print(term_docid_pairs[:5])
print('------------------------------------------------------')
print(term_docid_pairs[-5:])
print(chapter_length[:5])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\姚鑫浩\AppData\Local\Temp\jieba.cache
Loading model cost 1.312 seconds.
Prefix dict has been built successfully.


[('武道', 0), ('丹帝', 0), ('001', 0), ('圣人', 0), ('如天', 0)]
------------------------------------------------------
[('出自', 993), ('大儒', 993), ('之手', 993), ('一章', 993), ('一章', 993)]
[21.857255696757374, 19.621560803126833, 19.6347740288147, 18.68910440197632, 18.914766328687133]


In [4]:
from collections import defaultdict
# 构造倒排索引
# postings list中每一项为一个Posting类对象
class Posting(object):
    def __init__(self, docid, tf=0):
        self.docid = docid
        self.tf = tf
    def __repr__(self):
        return "<docid: %d, tf: %d>" % (self.docid, self.tf)
def abstract_jieba(term_docid_pairs):
    # 排序
    # 默认情况下两个tuple比较大小就是先按照第一个元素比较，再按照第二个元素比较，因此不需要用key=lambda x:...的方式定义排序方式。
    term_docid_pairs = sorted(term_docid_pairs)
    inverted_index = defaultdict(lambda: [Posting(-1, 0)])  
    for term, docid in term_docid_pairs:
        postings_list = inverted_index[term]
        if docid != postings_list[-1].docid:
            postings_list.append(Posting(docid, 1))
        else:
            postings_list[-1].tf += 1
    inverted_index = dict(inverted_index)
    return inverted_index
inverted_index = abstract_jieba(term_docid_pairs)

In [30]:
inverted_index['少主']

[<docid: -1, tf: 0>,
 <docid: 140, tf: 2>,
 <docid: 149, tf: 1>,
 <docid: 231, tf: 1>,
 <docid: 340, tf: 1>,
 <docid: 341, tf: 1>,
 <docid: 343, tf: 1>,
 <docid: 347, tf: 6>,
 <docid: 540, tf: 1>,
 <docid: 541, tf: 1>,
 <docid: 543, tf: 1>,
 <docid: 546, tf: 5>,
 <docid: 547, tf: 1>,
 <docid: 560, tf: 3>,
 <docid: 561, tf: 2>,
 <docid: 565, tf: 1>,
 <docid: 598, tf: 4>,
 <docid: 660, tf: 1>,
 <docid: 970, tf: 1>]

In [24]:
def frequency_making(all_terms_,inverted_index_,N):#做一个idf列表方便查询
    frequency=[]
    word=[]
    for term in all_terms_:
        word.append(term)
        frequency.append(N/(len(inverted_index_[term])-1))#-1是排除之前添加的（-1，x）的posting
    fre=np.array(frequency)
    fre=np.log(fre)
    frequency_dict_=dict(zip(word,fre))
    return frequency_dict_
N = len(js1)
frequency_dict=frequency_making(all_terms,inverted_index,N)#文本的idf
frequency_dict['天之骄子']

2.8134107167600364

In [36]:
data={"frequency_dict":frequency_dict,
              "chapter_length":chapter_length,
              "inverted_index":inverted_index,
              'all_chapters':all_chapters
              }
with open("chapter.pickle",'wb') as f:
    pickle.dump(data,f,pickle.HIGHEST_PROTOCOL)

In [5]:
with open(r"chapter.pickle",'rb') as f:
    data=pickle.load(f),
frequency_dict = data[0]['frequency_dict']
chapter_length = data[0]['chapter_length']
inverted_index = data[0]['inverted_index']
all_chapters = data[0]['all_chapters']

In [6]:
def get_postings_list(inverted_index, query_term):
    try:
        return inverted_index[query_term][1:]
    except KeyError:
        return []
def cosine_scores_TAAT(inverted_index, doc_length, query, k=3):
    scores = defaultdict(lambda: 0.0) # 保存分数
    query_terms = Counter(term for term in jieba.lcut_for_search(query)) # 对查询进行分词
    #print(query_terms)_Counter({'...': 1})
    query_len=[]
    for q in query_terms:
        if q in frequency_dict:
            idf=np.array(frequency_dict[q])
        else:
            continue
        w_tq = np.log(1 + query_terms[q])*idf
        #w_tq = (1+np.log( query_terms[q]))*idf
        query_len.append(w_tq)
        postings_list = get_postings_list(inverted_index, q)
        for posting in postings_list:
            w_td = np.log(1 + posting.tf)
            scores[posting.docid] += w_td * w_tq
    zc=np.array(query_len)
    lenth_=np.sqrt(np.sum(zc**2))#考虑idf后对向量的归一化
    results = [(score /(( doc_length[docid]+1)*lenth_), docid) for docid, score in scores.items()]
    sorted(results)[::-1]
    return results[0:k]
cosine_scores_TAAT(inverted_index, chapter_length, query='天才',k=10)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\姚鑫浩\AppData\Local\Temp\jieba.cache
Loading model cost 1.244 seconds.
Prefix dict has been built successfully.


[(-0.04806405034983983, 0),
 (-0.05327493389838503, 1),
 (-0.033591217407664575, 2),
 (-0.03548597850809849, 5),
 (-0.033721309605396474, 6),
 (-0.05428004645680651, 7),
 (-0.056077942609334366, 8),
 (-0.06480080862214901, 13),
 (-0.06804126509265944, 15),
 (-0.03416289287203419, 21)]

In [7]:
def retrieval_by_cosine_sim(inverted_index,abstract_length, data, query, k=20):
    top_scores = cosine_scores_TAAT(inverted_index, abstract_length, query, k=k)
    results = [ {'novel':data[docid]['novel'],'home':data[docid]['home'],'url':data[docid]['url'],'title':data[docid]['title']} for score,docid in top_scores]
    return results
def evaluate(query:str) -> list:
    url_list=retrieval_by_cosine_sim(inverted_index,chapter_length, all_chapters, query=query, k=10)
    return url_list
evaluate('废物')

[{'novel': '武道丹帝',
  'home': 'https://fanqienovel.com/page/6950489390904773646',
  'url': 'https://fanqienovel.com/reader/6950531579454161415',
  'title': '第004章 灵阁遇齐家'},
 {'novel': '武道丹帝',
  'home': 'https://fanqienovel.com/page/6950489390904773646',
  'url': 'https://fanqienovel.com/reader/6950531680717242888',
  'title': '第005章 一拳轰杀'},
 {'novel': '武道丹帝',
  'home': 'https://fanqienovel.com/page/6950489390904773646',
  'url': 'https://fanqienovel.com/reader/6950532653879656991',
  'title': '第009章 齐家谋划'},
 {'novel': '武碎星河',
  'home': 'https://fanqienovel.com/page/7064405896540982309',
  'url': 'https://fanqienovel.com/reader/7073299819586028032',
  'title': '第2章 血脉神通'},
 {'novel': '凡骨',
  'home': 'https://fanqienovel.com/page/7077516958534470656',
  'url': 'https://fanqienovel.com/reader/7082341353748857351',
  'title': '第7章 测灵骨，许太平无缘七峰'},
 {'novel': '太古神尊',
  'home': 'https://fanqienovel.com/page/6792789104695184392',
  'url': 'https://fanqienovel.com/reader/6792816859822424580',
  't